In [1]:
import pandas
from sqlalchemy import create_engine
import psycopg2
from datetime import date
from datetime import datetime, timedelta
import os

In [3]:
sql_general = """SELECT 
id, gridid_mun, covariable_id, date_occurrence, 
fecha_def, edad, sexo, neumonia, embarazo, 
diabetes, epoc, asma, inmusupr, hipertension, 
cardiovascular, obesidad, renal_cronica, tabaquismo, 
uci, intubado, tipo_paciente FROM occurrence
WHERE covariable_id in (5, 2, 3, 7) and date_occurrence >= '{0}' and date_occurrence < '{1}' """

dbuser = os.environ['DBUSER']
dbpass = os.environ['DBPASS']
dbport = os.environ['DBPORT']
dbhost = os.environ['DBHOST']

In [4]:
def get_occs_df(initial_date, final_date):
    dbname = 'epi_puma_covid19'
    conn_string = 'postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}:{dbport}/{dbname}'\
        .format(dbuser=dbuser, dbpass=dbpass, dbhost=dbhost, dbport=dbport, dbname=dbname)
    engine = create_engine(conn_string)
    sql_statement = sql_general.format(initial_date, final_date)
    #print(sql_statement)
    df = pd.read_sql(sql_statement, engine)
    return df

In [5]:
df = get_occs_df('2021-07-01', '2021-07-31')
df.head()

<IPython.core.display.Javascript object>

,id,gridid_mun,covariable_id,date_occurrence,fecha_def,edad,sexo,neumonia,embarazo,diabetes,...,asma,inmusupr,hipertension,cardiovascular,obesidad,renal_cronica,tabaquismo,uci,intubado,tipo_paciente
0,11939049,09007,3,2021-07-28,9999-99-99,29,MUJER,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO APLICA,NO APLICA,AMBULATORIO
1,7531684,27004,3,2021-07-06,9999-99-99,24,MUJER,NO,SI,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,HOSPITALIZADO
2,7531713,09006,3,2021-07-03,9999-99-99,75,MUJER,SI,NO,NO,...,NO,NO,SI,NO,SI,NO,NO,NO,NO,HOSPITALIZADO
3,7513700,07065,7,2021-07-02,9999-99-99,31,HOMBRE,NO,NO APLICA,NO,...,SI,NO,NO,NO,NO,NO,SI,NO APLICA,NO APLICA,AMBULATORIO
4,12106224,09008,3,2021-07-28,9999-99-99,68,MUJER,NO,NO,SI,...,NO,NO,NO,NO,NO,NO,NO,NO APLICA,NO APLICA,AMBULATORIO


In [9]:
df.columns

Index(['id', 'gridid_mun', 'covariable_id', 'date_occurrence', 'fecha_def',
       'edad', 'sexo', 'neumonia', 'embarazo', 'diabetes', 'epoc', 'asma',
       'inmusupr', 'hipertension', 'cardiovascular', 'obesidad',
       'renal_cronica', 'tabaquismo', 'uci', 'intubado', 'tipo_paciente'],
      dtype='object')

In [11]:
def classifier_dge(row):
    return row.tos == 'SI' and \
        (row.fiebre == 'SI' or row.cefalea == 'SI') and \
        (row.disnea == 'SI' or row.atralgias == 'SI' or row.mialgias == 'SI' or row.odinogia == 'SI' or
            row.rinorea == 'SI' or row.conjuntivitis == 'SI' or row.dolor_toracico == 'SI') 

In [ ]:
def classifier_nb(nb, ):
    

In [12]:
def edad2grooup(edad):
    if edad >= 60:
        return 'p_60ymas'
    elif edad >= 50 and edad <= 59:
        return 'p_50a59'
    elif edad >= 40 and edad <= 49:
        return 'p_40a49'
    elif edad >= 30 and edad <= 39:
        return 'p_30a39'
    elif edad >= 18 and edad <= 29:
        return 'p_18a29'
    else:
        return 'p_0a17'

In [ ]:
df = df[['sexo', 'edad', 'tos', 'fiebre', 'cefalea', 'disnea', 'atralgias', 'mialgias', 'odinogia', 'rinorea', 'conjuntivitis', 'dolor_toracico']]
df['edad'] = df['edad'].astype(int)
df['edad'] = df['edad'].apply(lambda x: edad2grooup(edad))
